In [ ]:
import pandas as pd
import requests
import os
import time

BASE_URL = "https://www.datos.gov.co/resource/rpmr-utcd.json"
DATA_PATH = "data/secop_completo.csv"

# Parámetros de descarga
LIMIT = 50000  # máximo permitido por Socrata
MAX_REGISTROS = 16000000  # máximo estimado en SECOP
PAUSA = 1  # segundos entre solicitudes para no saturar el servidor

def descargar_todos_los_datos():
    os.makedirs("data", exist_ok=True)
    offset = 0
    dfs = []

    print(f"⏳ Iniciando descarga masiva (hasta {MAX_REGISTROS:,} registros)...")

    while offset < MAX_REGISTROS:
        url = f"{BASE_URL}?$limit={LIMIT}&$offset={offset}"
        print(f"📥 Descargando lote desde offset {offset:,}...")

        try:
            response = requests.get(url)
            response.raise_for_status()

            data = response.json()
            if not data:  # si ya no hay más datos, detener el bucle
                print("✅ No hay más datos para descargar.")
                break

            df = pd.DataFrame(data)
            dfs.append(df)

            offset += LIMIT
            time.sleep(PAUSA)  # pequeña pausa para no saturar el servidor

        except requests.exceptions.RequestException as e:
            print(f"❌ Error en el offset {offset}: {e}")
            break

    # Unir todos los lotes en un solo DataFrame
    if dfs:
        df_final = pd.concat(dfs, ignore_index=True)
        df_final.to_csv(DATA_PATH, index=False)
        print(f"✅ Descarga completada: {len(df_final):,} registros guardados en '{DATA_PATH}'")
    else:
        print("⚠ No se descargaron datos.")

if __name__ == "__main__":
    descargar_todos_los_datos()


⏳ Iniciando descarga masiva (hasta 16,000,000 registros)...
📥 Descargando lote desde offset 0...
📥 Descargando lote desde offset 50,000...
📥 Descargando lote desde offset 100,000...
📥 Descargando lote desde offset 150,000...
📥 Descargando lote desde offset 200,000...
📥 Descargando lote desde offset 250,000...
📥 Descargando lote desde offset 300,000...
📥 Descargando lote desde offset 350,000...
📥 Descargando lote desde offset 400,000...
📥 Descargando lote desde offset 450,000...
📥 Descargando lote desde offset 500,000...
📥 Descargando lote desde offset 550,000...
📥 Descargando lote desde offset 600,000...
📥 Descargando lote desde offset 650,000...
📥 Descargando lote desde offset 700,000...
📥 Descargando lote desde offset 750,000...
📥 Descargando lote desde offset 800,000...
📥 Descargando lote desde offset 850,000...
📥 Descargando lote desde offset 900,000...
📥 Descargando lote desde offset 950,000...
📥 Descargando lote desde offset 1,000,000...
📥 Descargando lote desde offset 1,050,000.

In [2]:
import pandas as pd
import requests
import os
import time

BASE_URL = "https://www.datos.gov.co/resource/rpmr-utcd.json"
DATA_DIR = "data"
DATA_PATH = os.path.join(DATA_DIR, "secop_completo.parquet")

LIMIT = 50000        # máximo permitido por Socrata
MAX_REGISTROS = 16000000
PAUSA = 1            # segundos entre solicitudes


def get_offset_actual():
    """
    Calcula el offset actual según el tamaño del archivo Parquet.
    Si no existe, comienza desde 0.
    """
    if os.path.exists(DATA_PATH):
        df = pd.read_parquet(DATA_PATH)
        filas = len(df)
        print(f"✅ Archivo encontrado con {filas:,} registros. Reanudando desde offset {filas:,}...")
        return filas
    return 0


def guardar_incremental(df: pd.DataFrame, modo_append=True):
    """
    Guarda datos en formato Parquet de manera incremental.
    """
    if not os.path.exists(DATA_PATH) or not modo_append:
        df.to_parquet(DATA_PATH, index=False)
    else:
        df_existente = pd.read_parquet(DATA_PATH)
        df_final = pd.concat([df_existente, df], ignore_index=True)
        df_final.to_parquet(DATA_PATH, index=False)


def descargar_todos_los_datos():
    os.makedirs(DATA_DIR, exist_ok=True)

    offset = get_offset_actual()
    print(f"⏳ Iniciando descarga masiva... Guardando en {DATA_PATH}")

    while offset < MAX_REGISTROS:
        url = f"{BASE_URL}?$limit={LIMIT}&$offset={offset}"
        print(f"📥 Descargando lote desde offset {offset:,}...")

        try:
            response = requests.get(url)
            response.raise_for_status()
            data = response.json()

            if not data:
                print("✅ No hay más datos para descargar.")
                break

            df = pd.DataFrame(data)
            guardar_incremental(df, modo_append=True)

            offset += LIMIT
            time.sleep(PAUSA)

        except requests.exceptions.RequestException as e:
            print(f"❌ Error en offset {offset}: {e}")
            break

    print(f"✅ Descarga finalizada. Los datos completos están en '{DATA_PATH}'")


if __name__ == "__main__":
    descargar_todos_los_datos()


⏳ Iniciando descarga masiva... Guardando en data\secop_completo.parquet
📥 Descargando lote desde offset 0...


ImportError: Unable to find a usable engine; tried using: 'pyarrow', 'fastparquet'.
A suitable version of pyarrow or fastparquet is required for parquet support.
Trying to import the above resulted in these errors:
 - Missing optional dependency 'pyarrow'. pyarrow is required for parquet support. Use pip or conda to install pyarrow.
 - Missing optional dependency 'fastparquet'. fastparquet is required for parquet support. Use pip or conda to install fastparquet.